# Example script

This example showcases how to wrap you test to integrate them with pytest.

In [ ]:
import pytest

from giskard import Dataset, Model, Suite, demo
from giskard.testing import test_accuracy, test_f1

model_raw, df = demo.titanic()

wrapped_dataset = Dataset(
    name="Test Data Set",
    df=df,
    target="Survived",
    cat_columns=["Pclass", "Sex", "SibSp", "Parch", "Embarked"],
)

wrapped_model = Model(model=model_raw, model_type="classification", name="Classifier v1")

suite = (
    Suite(
        default_params={
            "model": wrapped_model,
            "dataset": wrapped_dataset,
        }
    )
    .add_test(test_f1(threshold=0.6))
    .add_test(test_accuracy(threshold=1))  # Certain to fail
)


@pytest.fixture
def dataset():
    return wrapped_dataset


@pytest.fixture
def model():
    return wrapped_model


# Single wrapped test
def test_only_accuracy(dataset, model):
    test_accuracy(model=model, dataset=dataset, threshold=1).assert_()


# Parametrise tests from suite
@pytest.mark.parametrize("test_partial", suite.to_unittest(), ids=lambda t: t.fullname)
def test_giskard(test_partial):
    test_partial.assert_()
